# Kalshi Deep Trading Bot - System Walkthrough

This notebook provides a comprehensive walkthrough of the Kalshi Deep Trading Bot system. Each section explains a core component and demonstrates how it works.

## Table of Contents

1. [System Overview](#1-system-overview)
2. [Configuration & Environment](#2-configuration--environment)
3. [Database Layer](#3-database-layer)
4. [Kalshi API Client](#4-kalshi-api-client)
5. [Research Pipeline (OpenAI)](#5-research-pipeline-openai)
6. [TrendRadar Signal Integration](#6-trendradar-signal-integration)
7. [Trading Decision Engine](#7-trading-decision-engine)
8. [Risk Management](#8-risk-management)
9. [Reconciliation & P&L](#9-reconciliation--pl)
10. [Performance Analysis](#10-performance-analysis)
11. [Full Pipeline Execution](#11-full-pipeline-execution)

---

## 1. System Overview

### Architecture

The Kalshi Deep Trading Bot is an automated prediction market trading system that:

```
+----------------+     +------------------+     +-------------------+
|   Kalshi API   |---->|   Trading Bot    |---->|    PostgreSQL     |
| (Market Data)  |     |   (Decisions)    |     |   (History)       |
+----------------+     +------------------+     +-------------------+
                              |
        +---------------------+---------------------+
        |                     |                     |
        v                     v                     v
+----------------+   +------------------+   +------------------+
|  OpenAI GPT-4o |   |  OpenAI GPT-5    |   |   TrendRadar     |
|  (Research)    |   |  (Decisions)     |   |   (News Signals) |
+----------------+   +------------------+   +------------------+
```

### Key Components

| Component | Purpose |
|-----------|----------|
| `trading_bot.py` | Main orchestration and pipeline |
| `kalshi_client.py` | RSA-authenticated API client |
| `research_client.py` | GPT-4o deep research |
| `trendradar_client.py` | News sentiment signals |
| `betting_models.py` | Pydantic decision models |
| `reconciliation.py` | Outcome tracking & P&L |
| `db/` | Database layer (SQLite/PostgreSQL) |

In [ ]:
# Initial Setup - Install required packages
# Run this cell first to ensure all dependencies are available

import sys
import os

# Add project root to path
project_root = os.path.dirname(os.path.abspath('.'))
if project_root not in sys.path:
    sys.path.insert(0, '.')

# Check Python version
print(f"Python version: {sys.version}")
print(f"Working directory: {os.getcwd()}")

# Verify key packages
packages = ['asyncio', 'httpx', 'pydantic', 'openai', 'rich']
for pkg in packages:
    try:
        __import__(pkg)
        print(f"  [OK] {pkg}")
    except ImportError:
        print(f"  [MISSING] {pkg} - run: uv sync")

---

## 2. Configuration & Environment

### How Configuration Works

The bot uses **Pydantic models** for type-safe configuration with validation. Configuration is loaded from:

1. **Environment variables** (`.env` file)
2. **Default values** (in `config.py`)

### Configuration Structure

```python
BotConfig
├── KalshiConfig      # API credentials, demo/live mode
├── OctagonConfig     # GPT-4o research API
├── OpenAIConfig      # GPT-5 decisions API
├── DatabaseConfig    # SQLite or PostgreSQL
├── TrendRadarConfig  # News signal settings
├── SchedulerConfig   # Trading intervals
└── Risk Parameters   # Bankroll, Kelly, limits
```

In [ ]:
# Load and inspect configuration
from config import BotConfig
from rich import print as rprint
from rich.table import Table
from rich.panel import Panel

# Load configuration from environment
config = BotConfig()

# Display configuration summary (hiding sensitive values)
table = Table(title="Bot Configuration")
table.add_column("Setting", style="cyan")
table.add_column("Value", style="green")

# Kalshi settings
table.add_row("Kalshi Mode", "DEMO" if config.kalshi.use_demo else "LIVE")
table.add_row("Kalshi API Key", config.kalshi.api_key[:8] + "..." if config.kalshi.api_key else "Not Set")

# Database settings
table.add_row("Database Type", config.database.db_type)
if config.database.db_type == "postgres":
    table.add_row("Database Host", config.database.pghost or "Not Set")
else:
    table.add_row("Database Path", config.database.db_path)

# Risk settings
table.add_row("Bankroll", f"${config.bankroll:,.2f}")
table.add_row("Max Bet Amount", f"${config.max_bet_amount:,.2f}")
table.add_row("Kelly Fraction", f"{config.kelly_fraction:.2f}")
table.add_row("Z-Threshold", f"{config.z_threshold:.2f}")
table.add_row("Max Daily Loss", f"${config.max_daily_loss:,.2f}")

# TrendRadar settings
table.add_row("TrendRadar Enabled", str(config.trendradar.enabled))

rprint(table)

In [ ]:
# Validate configuration
def validate_config(config: BotConfig) -> dict:
    """Validate configuration and return status."""
    checks = {}
    
    # Check Kalshi credentials
    checks["kalshi_api_key"] = bool(config.kalshi.api_key and not config.kalshi.api_key.startswith("your_"))
    checks["kalshi_private_key"] = bool(config.kalshi.private_key and "BEGIN RSA" in config.kalshi.private_key)
    
    # Check OpenAI
    checks["openai_api_key"] = bool(config.openai.api_key and config.openai.api_key.startswith("sk-"))
    
    # Check risk parameters
    checks["bankroll_positive"] = config.bankroll > 0
    checks["kelly_in_range"] = 0.1 <= config.kelly_fraction <= 1.5
    checks["max_bet_reasonable"] = config.max_bet_amount <= config.bankroll * 0.25
    
    return checks

validation = validate_config(config)
rprint(Panel("Configuration Validation", style="blue"))
for check, passed in validation.items():
    status = "[green]PASS[/green]" if passed else "[red]FAIL[/red]"
    rprint(f"  {status} {check}")

---

## 3. Database Layer

### Database Schema

The system uses either **SQLite** (local) or **PostgreSQL** (Neon cloud) for persistence.

#### Core Tables

| Table | Purpose |
|-------|----------|
| `betting_decisions` | All trading decisions with outcomes |
| `market_snapshots` | Point-in-time market data |
| `calibration_records` | Prediction vs actual tracking |
| `performance_daily` | Aggregated daily metrics |
| `run_history` | Bot execution history |

#### Key Fields in `betting_decisions`

```sql
-- Decision identifiers
decision_id, market_ticker, event_title

-- Trading action
action (buy_yes, buy_no, skip), bet_amount

-- Risk metrics
r_score, kelly_fraction, expected_return

-- Signal data
signal_applied, signal_direction, signal_strength

-- Outcome tracking
status (pending, settled), outcome, profit_loss
```

In [ ]:
# Connect to database and explore schema
import asyncio
from config import BotConfig

config = BotConfig()

async def explore_database():
    """Connect to database and show table information."""
    
    # Select appropriate database driver
    if config.database.db_type == "postgres":
        from db.postgres import PostgresDatabase
        db = PostgresDatabase()
    else:
        from db.database import Database
        db = Database(config.database.db_path)
    
    await db.connect()
    print(f"Connected to {config.database.db_type} database")
    
    # Get table counts
    tables = ["betting_decisions", "market_snapshots", "run_history", 
              "calibration_records", "performance_daily"]
    
    table = Table(title="Database Tables")
    table.add_column("Table", style="cyan")
    table.add_column("Row Count", style="green", justify="right")
    
    for tbl in tables:
        try:
            result = await db.execute(f"SELECT COUNT(*) FROM {tbl}")
            count = result[0][0]
            table.add_row(tbl, f"{count:,}")
        except Exception as e:
            table.add_row(tbl, f"[red]Error: {e}[/red]")
    
    await db.close()
    return table

# Run the async function
table = await explore_database()
rprint(table)

In [ ]:
# Query recent betting decisions
async def get_recent_decisions(limit: int = 10):
    """Fetch and display recent betting decisions."""
    
    if config.database.db_type == "postgres":
        from db.postgres import PostgresDatabase
        db = PostgresDatabase()
    else:
        from db.database import Database
        db = Database(config.database.db_path)
    
    await db.connect()
    
    query = """
        SELECT 
            market_ticker,
            action,
            bet_amount,
            r_score,
            signal_applied,
            signal_direction,
            status,
            profit_loss,
            timestamp
        FROM betting_decisions
        ORDER BY timestamp DESC
        LIMIT %s
    """ if config.database.db_type == "postgres" else """
        SELECT 
            market_ticker,
            action,
            bet_amount,
            r_score,
            signal_applied,
            signal_direction,
            status,
            profit_loss,
            timestamp
        FROM betting_decisions
        ORDER BY timestamp DESC
        LIMIT ?
    """
    
    result = await db.execute(query, (limit,))
    await db.close()
    
    # Display as table
    table = Table(title=f"Recent {limit} Decisions")
    table.add_column("Ticker", style="cyan", max_width=20)
    table.add_column("Action", style="yellow")
    table.add_column("Amount", justify="right")
    table.add_column("R-Score", justify="right")
    table.add_column("Signal", style="magenta")
    table.add_column("Status")
    table.add_column("P&L", justify="right")
    
    for row in result:
        ticker, action, amount, r_score, sig_applied, sig_dir, status, pnl, ts = row
        
        # Format values
        action_color = "green" if action == "buy_yes" else "red" if action == "buy_no" else "dim"
        amount_str = f"${amount:.2f}" if amount else "-"
        r_score_str = f"{r_score:.2f}" if r_score else "-"
        signal_str = sig_dir if sig_applied else "-"
        pnl_str = f"${pnl:.2f}" if pnl else "-"
        pnl_color = "green" if pnl and pnl > 0 else "red" if pnl and pnl < 0 else "dim"
        
        table.add_row(
            ticker[:20] if ticker else "-",
            f"[{action_color}]{action}[/{action_color}]",
            amount_str,
            r_score_str,
            signal_str,
            status or "-",
            f"[{pnl_color}]{pnl_str}[/{pnl_color}]"
        )
    
    return table

table = await get_recent_decisions(10)
rprint(table)

---

## 4. Kalshi API Client

### Authentication Flow

Kalshi uses **RSA signature authentication**:

1. Create message: `timestamp + method + path`
2. Sign with RSA private key (PSS padding, SHA256)
3. Include in headers: `KALSHI-ACCESS-KEY`, `KALSHI-ACCESS-TIMESTAMP`, `KALSHI-ACCESS-SIGNATURE`

### Key Endpoints

| Endpoint | Purpose |
|----------|----------|
| `GET /exchange/status` | Market hours & status |
| `GET /portfolio/balance` | Account balance |
| `GET /portfolio/positions` | Open positions |
| `GET /events` | Available events |
| `GET /markets/{ticker}` | Market details |
| `POST /portfolio/orders` | Place orders |

In [ ]:
# Initialize and test Kalshi client
from kalshi_client import KalshiClient
from config import BotConfig

config = BotConfig()
kalshi = KalshiClient(config.kalshi)

async def test_kalshi_connection():
    """Test Kalshi API connectivity."""
    
    results = {}
    
    # Test 1: Exchange status
    try:
        status = await kalshi.get_exchange_status()
        results["exchange_status"] = status
        print(f"[OK] Exchange Status: {status}")
    except Exception as e:
        results["exchange_status"] = None
        print(f"[FAIL] Exchange Status: {e}")
    
    # Test 2: Account balance
    try:
        balance = await kalshi.get_balance()
        results["balance"] = balance
        print(f"[OK] Account Balance: ${balance:.2f}")
    except Exception as e:
        results["balance"] = None
        print(f"[FAIL] Account Balance: {e}")
    
    # Test 3: Open positions
    try:
        positions = await kalshi.get_positions()
        results["positions"] = positions
        print(f"[OK] Open Positions: {len(positions) if positions else 0}")
    except Exception as e:
        results["positions"] = None
        print(f"[FAIL] Positions: {e}")
    
    return results

kalshi_results = await test_kalshi_connection()

In [ ]:
# Fetch top events by volume
async def get_top_events(limit: int = 10):
    """Fetch top events from Kalshi."""
    
    events = await kalshi.get_top_events(limit=limit)
    
    if not events:
        print("No events returned")
        return []
    
    table = Table(title=f"Top {limit} Events by Volume")
    table.add_column("Event", style="cyan", max_width=40)
    table.add_column("Ticker", style="yellow")
    table.add_column("Category", style="green")
    table.add_column("Volume (24h)", justify="right")
    
    for event in events[:limit]:
        table.add_row(
            event.get("title", "Unknown")[:40],
            event.get("event_ticker", "-"),
            event.get("category", "-"),
            f"${event.get('volume_24h', 0):,.0f}"
        )
    
    rprint(table)
    return events

events = await get_top_events(10)

In [ ]:
# Fetch markets for an event
async def get_event_markets(event_ticker: str):
    """Fetch markets for a specific event."""
    
    markets = await kalshi.get_markets_for_event(event_ticker)
    
    if not markets:
        print(f"No markets for event: {event_ticker}")
        return []
    
    table = Table(title=f"Markets for {event_ticker}")
    table.add_column("Market", style="cyan", max_width=50)
    table.add_column("Ticker", style="yellow")
    table.add_column("Yes Price", justify="right")
    table.add_column("No Price", justify="right")
    table.add_column("Volume", justify="right")
    
    for market in markets[:5]:  # Show top 5
        yes_price = market.get("yes_bid", 0) / 100  # Convert cents to dollars
        no_price = market.get("no_bid", 0) / 100
        
        table.add_row(
            market.get("title", "Unknown")[:50],
            market.get("ticker", "-"),
            f"{yes_price:.0%}",
            f"{no_price:.0%}",
            f"${market.get('volume', 0):,.0f}"
        )
    
    rprint(table)
    return markets

# Get markets for first event (if available)
if events:
    first_event = events[0]
    markets = await get_event_markets(first_event.get("event_ticker"))

---

## 5. Research Pipeline (OpenAI)

### Two-Stage Research Process

1. **GPT-4o Deep Research** (via Octagon API)
   - Comprehensive research on event topics
   - Returns markdown analysis with sources
   - Provides context for probability estimation

2. **GPT-5 Probability Extraction**
   - Structured output using Pydantic models
   - Extracts specific probability estimates
   - Returns `ProbabilityExtraction` object

### Pydantic Models

```python
class ProbabilityExtraction(BaseModel):
    probability: float  # 0.0 to 1.0
    confidence: float   # 0.0 to 1.0
    reasoning: str      # Explanation
```

In [ ]:
# Research client demonstration
from research_client import OctagonClient
from config import BotConfig

config = BotConfig()

# Note: This requires valid API key and will make real API calls
async def demonstrate_research(query: str):
    """Demonstrate the research pipeline."""
    
    print(f"Research Query: {query}")
    print("-" * 50)
    
    try:
        octagon = OctagonClient(config.octagon)
        
        # Perform research
        research = await octagon.research(query)
        
        if research:
            # Show first 500 chars of research
            preview = research[:500] + "..." if len(research) > 500 else research
            rprint(Panel(preview, title="Research Result", style="green"))
            return research
        else:
            print("No research returned")
            return None
            
    except Exception as e:
        print(f"Research error: {e}")
        return None

# Example research query
# Uncomment to run (uses API credits)
# research = await demonstrate_research("What is the probability Trump wins 2024 election?")

In [ ]:
# Probability extraction demonstration
from openai_utils import extract_probability
from betting_models import ProbabilityExtraction

async def demonstrate_probability_extraction(event_title: str, research_text: str):
    """Demonstrate probability extraction from research."""
    
    print(f"Event: {event_title}")
    print("-" * 50)
    
    try:
        # Extract probability using GPT-5
        extraction = await extract_probability(
            event_title=event_title,
            research=research_text,
            config=config.openai
        )
        
        if extraction:
            table = Table(title="Probability Extraction")
            table.add_column("Field", style="cyan")
            table.add_column("Value", style="green")
            
            table.add_row("Probability", f"{extraction.probability:.1%}")
            table.add_row("Confidence", f"{extraction.confidence:.1%}")
            table.add_row("Reasoning", extraction.reasoning[:100] + "...")
            
            rprint(table)
            return extraction
        else:
            print("Extraction failed")
            return None
            
    except Exception as e:
        print(f"Extraction error: {e}")
        return None

# Example (uses API credits)
# extraction = await demonstrate_probability_extraction(
#     "Will Bitcoin exceed $100,000 by end of 2024?",
#     "Based on current trends... [research text]"
# )

---

## 6. TrendRadar Signal Integration

### How TrendRadar Works

TrendRadar provides **news-based trading signals** by:

1. **Aggregating RSS feeds** from 12+ financial news sources
2. **Deduplicating** syndicated headlines (60% word overlap = same story)
3. **Analyzing sentiment** with negation-aware keyword detection
4. **Calculating signal strength** based on:
   - Source count and tier weighting (Tier 1: Reuters, Bloomberg, WSJ, AP)
   - Relevance to event (OpenAI embeddings)
   - Novelty (time decay)
   - Baseline frequency ratio

### Signal Influence on Decisions

| Signal Type | Confidence Adjustment | Kelly Multiplier |
|-------------|----------------------|------------------|
| Aligned (positive + buy_yes) | +15% max | 1.25x |
| Conflicting (positive + buy_no) | -10% | 0.8x |
| Neutral | 0% | 1.0x |

In [ ]:
# TrendRadar client demonstration
from trendradar_client import TrendRadarClient, TrendingSignal, calculate_signal_influence
from config import BotConfig

config = BotConfig()

async def demonstrate_trendradar():
    """Demonstrate TrendRadar signal fetching."""
    
    if not config.trendradar.enabled:
        print("TrendRadar is disabled in configuration")
        return None
    
    try:
        client = TrendRadarClient(config.trendradar)
        
        # Check health
        health = await client.health_check()
        print(f"TrendRadar Health: {health}")
        
        # Get signals for a topic
        signals = await client.get_signals_for_event(
            event_title="Federal Reserve Interest Rate Decision",
            keywords=["Fed", "interest rate", "Powell", "FOMC"]
        )
        
        if signals:
            table = Table(title="TrendRadar Signals")
            table.add_column("Topic", style="cyan")
            table.add_column("Sentiment", style="yellow")
            table.add_column("Strength", justify="right")
            table.add_column("Sources", justify="right")
            
            for signal in signals[:5]:
                sentiment_color = "green" if signal.sentiment == "positive" else "red" if signal.sentiment == "negative" else "dim"
                table.add_row(
                    signal.topic[:30],
                    f"[{sentiment_color}]{signal.sentiment}[/{sentiment_color}]",
                    f"{signal.strength:.2f}",
                    str(signal.unique_story_count)
                )
            
            rprint(table)
            return signals
        else:
            print("No signals returned")
            return []
            
    except Exception as e:
        print(f"TrendRadar error: {e}")
        return None

# signals = await demonstrate_trendradar()

In [ ]:
# Demonstrate signal influence calculation
from trendradar_client import TrendingSignal, calculate_signal_influence

def demonstrate_signal_influence():
    """Show how signals influence trading decisions."""
    
    # Create example signals
    signals = [
        TrendingSignal(
            topic="Fed Rate Cut",
            sentiment="positive",
            strength=0.8,
            unique_story_count=10,
            tier1_outlet_count=3
        ),
        TrendingSignal(
            topic="Economic Uncertainty",
            sentiment="negative",
            strength=0.6,
            unique_story_count=5,
            tier1_outlet_count=1
        ),
        TrendingSignal(
            topic="Market Update",
            sentiment="neutral",
            strength=0.3,
            unique_story_count=2,
            tier1_outlet_count=0
        ),
    ]
    
    table = Table(title="Signal Influence Examples")
    table.add_column("Signal", style="cyan")
    table.add_column("Sentiment")
    table.add_column("Action")
    table.add_column("Direction")
    table.add_column("Conf. Boost", justify="right")
    table.add_column("Kelly Mult", justify="right")
    
    for signal in signals:
        for action in ["buy_yes", "buy_no"]:
            influence = calculate_signal_influence(
                signal=signal,
                action=action,
                research_probability=0.65,
                config=config.trendradar
            )
            
            sentiment_color = "green" if signal.sentiment == "positive" else "red" if signal.sentiment == "negative" else "dim"
            direction_color = "green" if influence.direction == "aligned" else "red" if influence.direction == "conflicting" else "dim"
            
            table.add_row(
                signal.topic,
                f"[{sentiment_color}]{signal.sentiment}[/{sentiment_color}]",
                action,
                f"[{direction_color}]{influence.direction}[/{direction_color}]",
                f"{influence.confidence_boost:+.1%}",
                f"{influence.kelly_multiplier:.2f}x"
            )
    
    rprint(table)

demonstrate_signal_influence()

---

## 7. Trading Decision Engine

### Decision Flow

```
Research Probability ─┐
                      ├──> Calculate Edge ──> R-Score Check ──> Kelly Sizing ──> Decision
Market Price ─────────┘          │                  │                │
                                 │                  │                │
                            Edge = p - y       Skip if < Z     Cap at MAX_BET
```

### Key Calculations

| Metric | Formula | Purpose |
|--------|---------|----------|
| **Edge** | `research_prob - market_price` | Raw advantage |
| **R-Score** | `edge / sqrt(p * (1-p))` | Standardized edge (z-score) |
| **Kelly** | `(p - y) / (1 - y)` | Optimal position size |
| **Expected Return** | `(p - y) / y` | Expected profit per dollar |
| **Bet Size** | `bankroll * kelly_fraction * kelly` | Actual bet amount |

### Decision Actions

| Action | When |
|--------|------|
| `buy_yes` | R-score >= Z_THRESHOLD and research_prob > market_price |
| `buy_no` | R-score >= Z_THRESHOLD and research_prob < market_price |
| `skip` | R-score < Z_THRESHOLD or other constraints |

In [ ]:
# Trading calculations demonstration
import math

def calculate_trading_metrics(research_prob: float, market_price: float, bankroll: float, 
                              kelly_fraction: float, max_bet: float, z_threshold: float):
    """Calculate all trading metrics for a potential bet."""
    
    # Edge calculation
    edge = research_prob - market_price
    
    # R-score (z-score of edge)
    if 0 < market_price < 1:
        std_dev = math.sqrt(market_price * (1 - market_price))
        r_score = edge / std_dev
    else:
        r_score = 0
    
    # Kelly criterion
    if market_price < 1:
        kelly = (research_prob - market_price) / (1 - market_price)
    else:
        kelly = 0
    
    # Expected return
    if market_price > 0:
        expected_return = (research_prob - market_price) / market_price
    else:
        expected_return = 0
    
    # Bet sizing
    raw_bet = bankroll * kelly_fraction * max(kelly, 0)
    capped_bet = min(raw_bet, max_bet)
    
    # Decision
    if abs(r_score) < z_threshold:
        action = "skip"
        reason = f"R-score {r_score:.2f} < threshold {z_threshold}"
    elif edge > 0:
        action = "buy_yes"
        reason = f"Positive edge: {edge:.1%}"
    else:
        action = "buy_no"
        reason = f"Negative edge: {edge:.1%}"
    
    return {
        "research_prob": research_prob,
        "market_price": market_price,
        "edge": edge,
        "r_score": r_score,
        "kelly": kelly,
        "expected_return": expected_return,
        "raw_bet": raw_bet,
        "capped_bet": capped_bet,
        "action": action,
        "reason": reason
    }

# Example scenarios
scenarios = [
    (0.70, 0.50, "Strong positive edge"),
    (0.55, 0.50, "Weak positive edge"),
    (0.50, 0.50, "No edge"),
    (0.30, 0.50, "Strong negative edge"),
    (0.45, 0.50, "Weak negative edge"),
]

table = Table(title="Trading Decision Examples")
table.add_column("Scenario", style="cyan")
table.add_column("Research", justify="right")
table.add_column("Market", justify="right")
table.add_column("Edge", justify="right")
table.add_column("R-Score", justify="right")
table.add_column("Kelly", justify="right")
table.add_column("Bet", justify="right")
table.add_column("Action", style="yellow")

for research, market, desc in scenarios:
    metrics = calculate_trading_metrics(
        research_prob=research,
        market_price=market,
        bankroll=config.bankroll,
        kelly_fraction=config.kelly_fraction,
        max_bet=config.max_bet_amount,
        z_threshold=config.z_threshold
    )
    
    action_color = "green" if metrics["action"] == "buy_yes" else "red" if metrics["action"] == "buy_no" else "dim"
    
    table.add_row(
        desc,
        f"{research:.0%}",
        f"{market:.0%}",
        f"{metrics['edge']:+.1%}",
        f"{metrics['r_score']:.2f}",
        f"{metrics['kelly']:.2f}",
        f"${metrics['capped_bet']:.2f}",
        f"[{action_color}]{metrics['action']}[/{action_color}]"
    )

rprint(table)
print(f"\nConfig: Bankroll=${config.bankroll}, Kelly Fraction={config.kelly_fraction}, Z-Threshold={config.z_threshold}")

---

## 8. Risk Management

### Risk Controls

| Control | Setting | Purpose |
|---------|---------|----------|
| **Max Bet Amount** | $25 default | Limits single position size |
| **Kelly Fraction** | 0.5 default | Conservative sizing (half-Kelly) |
| **Z-Threshold** | 1.5 default | Minimum edge significance |
| **Max Daily Loss** | $100 default | Kill switch trigger |
| **Max Portfolio Positions** | 10 default | Concentration limit |
| **Skip Existing Positions** | true | Prevents doubling down |

### Kill Switch

The kill switch **halts all trading** when:
- Daily realized loss exceeds `MAX_DAILY_LOSS`
- OR daily loss percentage exceeds `MAX_DAILY_LOSS_PCT`

```python
if daily_pnl <= -max_daily_loss or daily_pnl <= -bankroll * max_daily_loss_pct:
    kill_switch_triggered = True
    # All decisions become "skip"
```

In [ ]:
# Risk management demonstration
from config import BotConfig

config = BotConfig()

def analyze_risk_settings():
    """Analyze current risk management settings."""
    
    table = Table(title="Risk Management Settings")
    table.add_column("Setting", style="cyan")
    table.add_column("Value", style="green")
    table.add_column("Impact", style="yellow")
    
    # Bankroll
    table.add_row(
        "Bankroll",
        f"${config.bankroll:,.2f}",
        "Total capital at risk"
    )
    
    # Max bet
    max_bet_pct = (config.max_bet_amount / config.bankroll) * 100
    table.add_row(
        "Max Bet Amount",
        f"${config.max_bet_amount:.2f} ({max_bet_pct:.1f}%)",
        "Maximum single position"
    )
    
    # Kelly fraction
    kelly_desc = "Conservative" if config.kelly_fraction <= 0.5 else "Moderate" if config.kelly_fraction <= 1.0 else "Aggressive"
    table.add_row(
        "Kelly Fraction",
        f"{config.kelly_fraction:.2f} ({kelly_desc})",
        "Position sizing multiplier"
    )
    
    # Z-threshold
    z_desc = "Strict" if config.z_threshold >= 2.0 else "Moderate" if config.z_threshold >= 1.0 else "Loose"
    table.add_row(
        "Z-Threshold",
        f"{config.z_threshold:.2f} ({z_desc})",
        "Minimum edge significance"
    )
    
    # Kill switch
    table.add_row(
        "Kill Switch",
        "Enabled" if config.enable_kill_switch else "Disabled",
        "Auto-halt on max loss"
    )
    
    table.add_row(
        "Max Daily Loss",
        f"${config.max_daily_loss:.2f} ({config.max_daily_loss_pct*100:.0f}%)",
        "Kill switch trigger"
    )
    
    # Portfolio limit
    table.add_row(
        "Max Positions",
        str(config.max_portfolio_positions),
        "Concentration limit"
    )
    
    rprint(table)
    
    # Risk analysis
    print("\n--- Risk Analysis ---")
    
    # Worst case: all positions lose
    worst_case = config.max_bet_amount * config.max_portfolio_positions
    print(f"Worst case loss (all positions): ${worst_case:.2f} ({worst_case/config.bankroll*100:.1f}% of bankroll)")
    
    # Expected bets to hit kill switch
    if config.max_bet_amount > 0:
        bets_to_kill = config.max_daily_loss / config.max_bet_amount
        print(f"Consecutive losses to trigger kill switch: {bets_to_kill:.0f} bets")

analyze_risk_settings()

In [ ]:
# Kill switch simulation
def simulate_kill_switch(daily_pnl: float, bankroll: float, max_loss: float, max_loss_pct: float):
    """Simulate kill switch behavior."""
    
    absolute_trigger = daily_pnl <= -max_loss
    percentage_trigger = daily_pnl <= -bankroll * max_loss_pct
    
    triggered = absolute_trigger or percentage_trigger
    
    return {
        "daily_pnl": daily_pnl,
        "absolute_trigger": absolute_trigger,
        "percentage_trigger": percentage_trigger,
        "kill_switch_triggered": triggered
    }

# Simulate various P&L scenarios
scenarios = [-150, -100, -80, -50, 0, 50, 100]

table = Table(title="Kill Switch Simulation")
table.add_column("Daily P&L", justify="right")
table.add_column("Absolute Trigger", justify="center")
table.add_column("% Trigger", justify="center")
table.add_column("Kill Switch", justify="center")

for pnl in scenarios:
    result = simulate_kill_switch(
        daily_pnl=pnl,
        bankroll=config.bankroll,
        max_loss=config.max_daily_loss,
        max_loss_pct=config.max_daily_loss_pct
    )
    
    pnl_color = "red" if pnl < 0 else "green" if pnl > 0 else "dim"
    abs_status = "[red]YES[/red]" if result["absolute_trigger"] else "[green]NO[/green]"
    pct_status = "[red]YES[/red]" if result["percentage_trigger"] else "[green]NO[/green]"
    kill_status = "[bold red]TRIGGERED[/bold red]" if result["kill_switch_triggered"] else "[green]OK[/green]"
    
    table.add_row(
        f"[{pnl_color}]${pnl:+,.2f}[/{pnl_color}]",
        abs_status,
        pct_status,
        kill_status
    )

rprint(table)
print(f"\nThresholds: ${config.max_daily_loss} absolute, {config.max_daily_loss_pct*100:.0f}% of ${config.bankroll:,.0f}")

---

## 9. Reconciliation & P&L

### Reconciliation Process

The `ReconciliationEngine` automatically tracks bet outcomes:

1. **Fetch pending decisions** from database
2. **Query Kalshi API** for settled markets
3. **Match outcomes** to decisions
4. **Calculate P&L**:
   - `buy_yes` wins if `outcome = 'yes'`
   - `buy_no` wins if `outcome = 'no'`
   - Profit = `payout - bet_amount` (win) or `-bet_amount` (loss)
5. **Update database** with outcome and P&L

### P&L Calculation

```python
# For buy_yes at price 0.60 ($6 bet wins $10 if yes)
if outcome == "yes":
    payout = bet_amount / price  # $6 / 0.6 = $10
    profit = payout - bet_amount  # $10 - $6 = $4
else:
    profit = -bet_amount  # -$6
```

In [ ]:
# P&L calculation demonstration
def calculate_pnl(action: str, bet_amount: float, price: float, outcome: str) -> dict:
    """Calculate P&L for a settled bet."""
    
    # Determine if we won
    if action == "buy_yes":
        won = outcome == "yes"
    elif action == "buy_no":
        won = outcome == "no"
    else:
        return {"profit_loss": 0, "won": None, "payout": 0}
    
    # Calculate payout and profit
    if won:
        # Winner gets $1 per contract, we bought at price
        contracts = bet_amount / price
        payout = contracts  # $1 per contract
        profit = payout - bet_amount
    else:
        payout = 0
        profit = -bet_amount
    
    return {
        "action": action,
        "bet_amount": bet_amount,
        "price": price,
        "outcome": outcome,
        "won": won,
        "payout": payout,
        "profit_loss": profit,
        "roi": (profit / bet_amount) * 100 if bet_amount > 0 else 0
    }

# Example scenarios
scenarios = [
    ("buy_yes", 10.00, 0.60, "yes"),   # Win at 60%
    ("buy_yes", 10.00, 0.60, "no"),    # Lose at 60%
    ("buy_no", 10.00, 0.40, "no"),     # Win at 40%
    ("buy_no", 10.00, 0.40, "yes"),    # Lose at 40%
    ("buy_yes", 25.00, 0.30, "yes"),   # Big win at 30%
]

table = Table(title="P&L Calculation Examples")
table.add_column("Action", style="cyan")
table.add_column("Bet", justify="right")
table.add_column("Price", justify="right")
table.add_column("Outcome")
table.add_column("Result")
table.add_column("Payout", justify="right")
table.add_column("P&L", justify="right")
table.add_column("ROI", justify="right")

for action, bet, price, outcome in scenarios:
    result = calculate_pnl(action, bet, price, outcome)
    
    result_str = "[green]WIN[/green]" if result["won"] else "[red]LOSS[/red]"
    pnl_color = "green" if result["profit_loss"] > 0 else "red"
    
    table.add_row(
        action,
        f"${bet:.2f}",
        f"{price:.0%}",
        outcome,
        result_str,
        f"${result['payout']:.2f}",
        f"[{pnl_color}]${result['profit_loss']:+.2f}[/{pnl_color}]",
        f"{result['roi']:+.0f}%"
    )

rprint(table)

In [ ]:
# Query actual P&L from database
async def get_pnl_summary():
    """Get P&L summary from database."""
    
    if config.database.db_type == "postgres":
        from db.postgres import PostgresDatabase
        db = PostgresDatabase()
    else:
        from db.database import Database
        db = Database(config.database.db_path)
    
    await db.connect()
    
    # Overall P&L
    query = """
        SELECT 
            COUNT(*) as total_bets,
            SUM(CASE WHEN profit_loss > 0 THEN 1 ELSE 0 END) as wins,
            SUM(CASE WHEN profit_loss < 0 THEN 1 ELSE 0 END) as losses,
            SUM(bet_amount) as total_wagered,
            SUM(profit_loss) as total_pnl
        FROM betting_decisions
        WHERE status = 'settled' AND profit_loss IS NOT NULL
    """
    
    result = await db.execute(query)
    await db.close()
    
    if result and result[0][0] > 0:
        total, wins, losses, wagered, pnl = result[0]
        win_rate = (wins / total) * 100 if total > 0 else 0
        roi = (pnl / wagered) * 100 if wagered and wagered > 0 else 0
        
        table = Table(title="P&L Summary")
        table.add_column("Metric", style="cyan")
        table.add_column("Value", style="green", justify="right")
        
        table.add_row("Total Settled Bets", str(total))
        table.add_row("Wins", str(wins))
        table.add_row("Losses", str(losses))
        table.add_row("Win Rate", f"{win_rate:.1f}%")
        table.add_row("Total Wagered", f"${wagered:,.2f}" if wagered else "$0")
        
        pnl_color = "green" if pnl and pnl > 0 else "red" if pnl and pnl < 0 else "dim"
        table.add_row("Total P&L", f"[{pnl_color}]${pnl:+,.2f}[/{pnl_color}]" if pnl else "$0")
        table.add_row("ROI", f"{roi:+.1f}%")
        
        rprint(table)
    else:
        print("No settled bets found in database")

await get_pnl_summary()

---

## 10. Performance Analysis

### Key Performance Metrics

| Metric | Description |
|--------|-------------|
| **Win Rate** | Percentage of winning bets |
| **ROI** | Return on investment (profit / wagered) |
| **Sharpe Ratio** | Risk-adjusted return |
| **Max Drawdown** | Largest peak-to-trough decline |
| **Brier Score** | Calibration accuracy (lower = better) |
| **R-Score Effectiveness** | Win rate by R-score bucket |

### Calibration Analysis

Good calibration means:
- When we predict 70%, events happen ~70% of the time
- Brier score close to 0 indicates accurate predictions
- Calibration error shows systematic over/under-confidence

In [ ]:
# Performance analysis
async def analyze_performance():
    """Comprehensive performance analysis."""
    
    if config.database.db_type == "postgres":
        from db.postgres import PostgresDatabase
        db = PostgresDatabase()
    else:
        from db.database import Database
        db = Database(config.database.db_path)
    
    await db.connect()
    
    # R-score effectiveness
    r_score_query = """
        SELECT 
            CASE 
                WHEN ABS(r_score) < 1.0 THEN '0-1'
                WHEN ABS(r_score) < 1.5 THEN '1-1.5'
                WHEN ABS(r_score) < 2.0 THEN '1.5-2'
                ELSE '2+'
            END as r_score_bucket,
            COUNT(*) as count,
            SUM(CASE WHEN profit_loss > 0 THEN 1 ELSE 0 END) as wins,
            AVG(profit_loss) as avg_pnl
        FROM betting_decisions
        WHERE status = 'settled' AND r_score IS NOT NULL
        GROUP BY r_score_bucket
        ORDER BY r_score_bucket
    """
    
    result = await db.execute(r_score_query)
    
    if result:
        table = Table(title="R-Score Effectiveness")
        table.add_column("R-Score Bucket", style="cyan")
        table.add_column("Count", justify="right")
        table.add_column("Win Rate", justify="right")
        table.add_column("Avg P&L", justify="right")
        
        for bucket, count, wins, avg_pnl in result:
            win_rate = (wins / count) * 100 if count > 0 else 0
            pnl_color = "green" if avg_pnl and avg_pnl > 0 else "red"
            
            table.add_row(
                bucket,
                str(count),
                f"{win_rate:.1f}%",
                f"[{pnl_color}]${avg_pnl:+.2f}[/{pnl_color}]" if avg_pnl else "$0"
            )
        
        rprint(table)
    
    # Signal effectiveness
    signal_query = """
        SELECT 
            COALESCE(signal_direction, 'no_signal') as direction,
            COUNT(*) as count,
            SUM(CASE WHEN profit_loss > 0 THEN 1 ELSE 0 END) as wins,
            SUM(profit_loss) as total_pnl
        FROM betting_decisions
        WHERE status = 'settled'
        GROUP BY signal_direction
    """
    
    result = await db.execute(signal_query)
    
    if result:
        table = Table(title="Signal Effectiveness")
        table.add_column("Signal Direction", style="cyan")
        table.add_column("Count", justify="right")
        table.add_column("Win Rate", justify="right")
        table.add_column("Total P&L", justify="right")
        
        for direction, count, wins, pnl in result:
            win_rate = (wins / count) * 100 if count > 0 else 0
            pnl_color = "green" if pnl and pnl > 0 else "red"
            dir_color = "green" if direction == "aligned" else "red" if direction == "conflicting" else "dim"
            
            table.add_row(
                f"[{dir_color}]{direction}[/{dir_color}]",
                str(count),
                f"{win_rate:.1f}%",
                f"[{pnl_color}]${pnl:+.2f}[/{pnl_color}]" if pnl else "$0"
            )
        
        rprint(table)
    
    await db.close()

await analyze_performance()

---

## 11. Full Pipeline Execution

### Running the Trading Bot

The complete pipeline executes in this order:

1. **Fetch Events** - Get top events by 24h volume
2. **Get Markets** - Fetch markets for each event
3. **Filter Positions** - Skip events with existing positions
4. **Fetch Signals** - Get TrendRadar news signals (if enabled)
5. **Research Events** - GPT-4o deep research (parallel batch)
6. **Extract Probabilities** - GPT-5 structured extraction
7. **Get Market Odds** - Fetch current bid/ask prices
8. **Generate Decisions** - Calculate metrics and decide
9. **Apply Signal Influence** - Adjust for news signals
10. **Risk Checks** - Kill switch, position limits
11. **Save to Database** - Persist all decisions
12. **Place Bets** - Execute orders (if live mode)

### CLI Commands

```bash
# Dry run (no real bets)
uv run trading-bot

# Live trading
uv run trading-bot --live

# With options
uv run trading-bot --max-expiration-hours 6 --max-events 20

# Reconcile outcomes
uv run trading-bot --reconcile

# Show stats
uv run trading-bot --stats
```

In [ ]:
# Run the trading bot pipeline (dry run)
from trading_bot import SimpleTradingBot
from config import BotConfig

config = BotConfig()

async def run_bot_dry_run(max_events: int = 5):
    """Run the trading bot in dry-run mode."""
    
    print("="*60)
    print("TRADING BOT - DRY RUN")
    print("="*60)
    print(f"Mode: {'DEMO' if config.kalshi.use_demo else 'LIVE'}")
    print(f"Max Events: {max_events}")
    print(f"TrendRadar: {'Enabled' if config.trendradar.enabled else 'Disabled'}")
    print("="*60)
    
    # Initialize bot
    bot = SimpleTradingBot(
        config=config,
        dry_run=True  # Always dry run in notebook
    )
    
    # Run the pipeline
    try:
        decisions = await bot.run(
            max_events=max_events,
            max_expiration_hours=24
        )
        
        print(f"\nGenerated {len(decisions)} decisions")
        
        # Display decisions
        if decisions:
            table = Table(title="Trading Decisions")
            table.add_column("Market", style="cyan", max_width=30)
            table.add_column("Action", style="yellow")
            table.add_column("Amount", justify="right")
            table.add_column("R-Score", justify="right")
            table.add_column("Edge", justify="right")
            table.add_column("Signal")
            
            for d in decisions[:10]:  # Show first 10
                action_color = "green" if d.action == "buy_yes" else "red" if d.action == "buy_no" else "dim"
                signal_str = d.signal_direction if d.signal_applied else "-"
                
                table.add_row(
                    d.market_ticker[:30] if d.market_ticker else "-",
                    f"[{action_color}]{d.action}[/{action_color}]",
                    f"${d.bet_amount:.2f}" if d.bet_amount else "-",
                    f"{d.r_score:.2f}" if d.r_score else "-",
                    f"{d.edge:+.1%}" if d.edge else "-",
                    signal_str
                )
            
            rprint(table)
        
        return decisions
        
    except Exception as e:
        print(f"Error running bot: {e}")
        import traceback
        traceback.print_exc()
        return []

# Uncomment to run (uses API credits)
# decisions = await run_bot_dry_run(max_events=3)

In [ ]:
# Run reconciliation
from reconciliation import ReconciliationEngine

async def run_reconciliation():
    """Run reconciliation to update P&L for settled bets."""
    
    print("Running reconciliation...")
    
    try:
        engine = ReconciliationEngine(config)
        results = await engine.reconcile()
        
        print(f"\nReconciliation Results:")
        print(f"  Pending decisions checked: {results.get('pending_count', 0)}")
        print(f"  Settled: {results.get('settled_count', 0)}")
        print(f"  P&L updated: {results.get('pnl_updated', 0)}")
        
        return results
        
    except Exception as e:
        print(f"Reconciliation error: {e}")
        return None

# Uncomment to run
# results = await run_reconciliation()

---

## Summary

### System Components Covered

| Section | Component | Status |
|---------|-----------|--------|
| 2 | Configuration | Validated |
| 3 | Database | Connected |
| 4 | Kalshi API | Authenticated |
| 5 | Research Pipeline | Ready |
| 6 | TrendRadar | Configured |
| 7 | Decision Engine | Functional |
| 8 | Risk Management | Active |
| 9 | Reconciliation | Ready |
| 10 | Performance Analysis | Available |
| 11 | Full Pipeline | Executable |

### Next Steps

1. **Validate Production Readiness**: Run `uv run python validate_production.py`
2. **Test in Demo Mode**: Run `uv run trading-bot` for dry-run testing
3. **Monitor Dashboard**: Start `uv run python dashboard/api.py`
4. **Go Live**: Run `uv run trading-bot --live` when ready

---

*Generated for Kalshi Deep Trading Bot v1.0*